In [2]:
import altair as alt
# import streamlit as st
import pandas as pd
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
import numpy as np
from vega_datasets import data
import streamlit as st
from sklearn.linear_model import LinearRegression

alt.data_transformers.disable_max_rows()

# @st.cache
def get_energy(with_coords=True):

    energy_sources = pd.read_csv('./data/primary-energy-source-bar.csv')

    energy_sources = energy_sources.sort_values(['Entity', 'Year'])

    for entity in energy_sources.Entity.unique():
        indices = energy_sources.Entity == entity
        for col in energy_sources.columns:
            if col != 'Entity' and col != 'Code' and col != 'Year' and col != 'TotalEnergy':
                temp = energy_sources[indices][col]
                temp = temp.fillna('ffill')
                energy_sources.loc[indices, col] = temp

    energy_sources = energy_sources[energy_sources.Year == 2019].copy()
    energy_sources['TotalEnergy'] = 0
    for col in energy_sources.columns:
        if col != 'Entity' and col != 'Code' and col != 'Year' and col != 'TotalEnergy':
            energy_sources['TotalEnergy'] += energy_sources[col]
    if with_coords == False:
        return energy_sources
    lats = pd.read_csv('./data/average-latitude-longitude-countries.csv')

    energy_sources = energy_sources[energy_sources.Entity != 'Europe']
    country_lat = energy_sources.merge(lats, left_on='Entity', right_on='Country')
    return country_lat

# @st.cache
def get_energypc(with_coords=True):
    population = pd.read_csv('./data/population-past-future.csv')
    population = population[~population.Code.isnull()]
    population = population[population.Code != 'NaN']
    population = population.sort_values(by=['Entity', 'Year'])
    for entity in population.Entity.unique():
        indices = population.Entity == entity
        temp = population[indices]['Population (historical estimates and future projections)']
        temp = temp.fillna('ffill')
        population.loc[indices, 'Population (historical estimates and future projections)'] = temp

    population = population[population.Year == 2019]
    population['Entity'] = population.Entity.astype('str')
    population = population[population.Entity != 'Europe']

    df = get_energy(with_coords)
    df['Entity'] = df.Entity.astype('str')
    joined = population.merge(df, on='Entity')
    joined['EnergyPerCapita'] = joined['TotalEnergy']/joined['Population (historical estimates and future projections)']
    if with_coords:
        joined['Latitude_x'] = joined['Latitude']
        joined['Longitude_x'] = joined['Longitude']
    return joined

# @st.cache
def get_fuelco2():
    fuel_co2 = pd.read_csv('./data/co2-emissions-by-fuel-line.csv')
    fuel_co2 = fuel_co2[(fuel_co2.Year == 2019)]
    co2_cols = ['Annual CO2 emissions from oil', 'Annual CO2 emissions from flaring', 'Annual CO2 emissions from cement', 'Annual CO2 emissions from coal', 'Annual CO2 emissions from gas', 'Annual CO2 emissions from other industry']
    co2_rename = ['Oil', 'Flaring', 'Cement', 'Coal', 'Gas', 'Other Industries']
    fuel_co2 = fuel_co2.rename(index=str, columns=dict(zip(co2_cols, co2_rename)))
    fuel_co2['NetCO2'] = 0
    for col in co2_rename:
        fuel_co2[col] = fuel_co2[col].fillna(0)
        fuel_co2['NetCO2'] += fuel_co2[col]
    
    co2_cols = co2_rename
    population = pd.read_csv('./data/population-past-future.csv')
    population = population[~population.Code.isnull()]
    population = population[population.Code != 'NaN']
    population = population[population.Year == 2019]
    population['Entity'] = population.Entity.astype('str')
    population = population[population.Entity != 'Europe']

    fuel_co2['Entity'] = fuel_co2.Entity.astype('str')
    co2_pc = population.merge(fuel_co2, on='Entity')
    co2_pc['NetCO2_PC'] = co2_pc['NetCO2']/co2_pc['Population (historical estimates and future projections)']
    for col in co2_cols:
        co2_pc[col + '_PC'] = co2_pc[col]/co2_pc['Population (historical estimates and future projections)']
    # co2_pc.head()

    world_population = 7713468203
    for col in co2_cols+['NetCO2']:
        co2_pc[col + '_WORLD'] = co2_pc[col+'_PC']*world_population
    return co2_pc, co2_cols

# @st.cache
def get_energy_charts_circle():
    url = 'https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json'
    selector = alt.selection_single(name="selector", fields=['Entity'])
    countries = alt.topo_feature(data.world_110m.url, 'countries')
    geo = alt.Chart(countries).mark_geoshape(
        fill='lightblue',
        stroke='white').properties(
        width=580,
        height=500
    )
    country_lat = get_energy()

    color_range = alt.Color('TotalEnergy:Q', 
                        scale=alt.Scale(
                            type='pow', exponent=0.6,
    #                         domain=(temp_min, temp_max), 
                            range=['yellow', 'orange', 'red']
                        ))
    circles = alt.Chart(country_lat).mark_circle().encode(
        latitude='Latitude:Q',
        longitude="Longitude:Q",
        size=alt.Size("TotalEnergy:Q", scale=alt.Scale(range=[0, 3000]), legend=None),
        tooltip=['Entity'],
        color=alt.condition(selector, color_range, alt.value('lightgreen'))
    ).add_selection(selector)

    g1 = geo+circles

    joined_lats = get_energypc()

    countries = alt.topo_feature(url, 'countries')
    geo2 = alt.Chart(countries).mark_geoshape(
            fill='lightblue',
            stroke='white'
        ).properties(
            width=580,
            height=500
    ).project(
        clipExtent= [[0, 0], [580, 360]]
    )
    color_range2 = alt.Color('EnergyPerCapita:Q', 
                        scale=alt.Scale(
                            type='pow', exponent=0.6,
    #                         domain=(temp_min, temp_max), 
                            range=['yellow', 'orange', 'red']
                        )
                    )
    circles2 = alt.Chart(joined_lats).mark_circle().encode(
        latitude='Latitude:Q',
        longitude="Longitude:Q",
        size=alt.Size("EnergyPerCapita:Q", scale=alt.Scale(range=[0, 1000]), legend=None),
        tooltip=['Entity', 'EnergyPerCapita:Q'],
        color=alt.condition(selector, color_range2, alt.value('lightgreen'))
        
    ).add_selection(selector)

    g2 = (geo2+circles2)

    return (g1&g2).resolve_scale(color='independent'), selector
    # return g2, selector


# @st.cache
def get_energypc_chart_heatmap():
    url = 'https://cdn.jsdelivr.net/npm/world-atlas@2/countries-110m.json'
    selector = alt.selection_single(name="selector", fields=['Entity'])
    countries = alt.topo_feature(url, 'countries')
    energy_sources = get_energy(with_coords=False)
    energy_sources.loc[(energy_sources.Entity == 'United States'), ['Entity']] = 'United States of America'
    color_range = alt.Color('TotalEnergy:Q', 
                       scale=alt.Scale(
                        type='pow', exponent=0.6, 
                        range=['yellow', 'orange', 'red']
                    ))
    base_chart = alt.Chart(countries,
        title='Net Energy Consumption by Country'
    ).mark_geoshape(stroke='white').encode(
        color=alt.condition(selector, color_range, 
                    alt.value('lightgray')
        ),
        # color = color_range,
        tooltip = [alt.Tooltip('properties.name:N', title='Country'), alt.Tooltip('TotalEnergy:Q', title='Total Energy Consumed (TWh)')]
    ).transform_lookup(
        lookup='properties.name',
        from_=alt.LookupData(energy_sources, 'Entity', ['TotalEnergy', 'Country', 'Entity'])
    ).properties(
        width=580,
        height=500
    ).add_selection(selector)

    energy_pc = get_energypc(with_coords=False)
    energy_pc.loc[(energy_pc.Entity == 'United States'), ['Entity']] = 'United States of America'
    pc_chart = alt.Chart(countries,
        title='Per Capita Energy Consumption by Country'
    ).mark_geoshape(stroke='white').encode(
        color=alt.condition(selector, alt.Color('EnergyPerCapita:Q', 
                       scale=alt.Scale(
                        type='pow', exponent=0.6, 
                        range=['yellow', 'orange', 'red']
                    )), 
                    alt.value('lightgray')
        ),
        tooltip = [alt.Tooltip('properties.name:N', title='Country'), alt.Tooltip('EnergyPerCapita:Q', title='Per Capita Energy Consumed (TWh/person)')]
    ).transform_lookup(
        lookup='properties.name',
        from_=alt.LookupData(energy_pc, 'Entity', ['EnergyPerCapita', 'Country', 'Entity'])
    ).properties(
        width=580,
        height=500
    ).add_selection(selector)

    # return (base_chart|pc_chart).resolve_scale(color='independent'), selector
    return base_chart, pc_chart, selector

"""

## Energy Consumption and CO2 Emissions

"""


"""
Energy production is the major driving force behind climate change. Energy production is driven by energy needs which can vary by country.
"""
geo_map1, geo_map2, selector = get_energypc_chart_heatmap()

# per_capita = st.checkbox("Show Per Capita Energy")

In [7]:
co2_pc, co2_cols = get_fuelco2()
co2_pc.loc[(co2_pc.Entity == 'United States'), ['Entity']] = 'United States of America'
# co2_pc_world = co2_pc[co2_pc.Entity == 'World']
net_world_co2 = co2_pc[co2_pc.Entity == 'World'].NetCO2_WORLD.max()
co2_pc = co2_pc[co2_pc.Entity != 'World']
fuel_co2_long_2 = co2_pc.melt(id_vars='Entity', value_vars=list(map(lambda x: x + "_WORLD", co2_cols)))

co2_chart = alt.Chart(
    fuel_co2_long_2,
    title='CO2 Emissions by Fuel Type Based on Selected Country\'s Rate'
).mark_bar().encode(
    x=alt.X('variable', title='Emissions Source'), 
    y=alt.Y('value:Q', title='CO2 emissions (T)'),
    tooltip = [alt.Tooltip('value:Q', title='CO2 emissions (T)')]
).properties(
        width=200,
        height=200
).transform_filter(
            selector
).add_selection(selector)

netCO2_Line = pd.DataFrame({
    'CO2 emissions(T)': [net_world_co2],
    'Type': "Original Net CO2 Emissions (World)"
    })
netCO2_rule_chart = alt.Chart(netCO2_Line).mark_rule().encode(
    y=alt.Y('CO2 emissions(T):Q', title='CO2 emissions(T)'),
    color = alt.Color('Type', scale = alt.Scale(range=['red']), legend=None)
).properties(
    width=200,
    height=200
)

fuel_co2_long_full = co2_pc.melt(id_vars='Entity', value_vars=['NetCO2_WORLD'])
co2_chart_full = alt.Chart(
    fuel_co2_long_full,
    title='Net Global CO2 Emissions Based on Selected Country\'s Rate'
).mark_bar().encode(
    x=alt.X('variable', title='World'), 
    y=alt.Y('value:Q', title='CO2 emissions(T)'),
    tooltip = [alt.Tooltip('value:Q', title='CO2 emissions (T)')]
).properties(
        width=200,
        height=200
).transform_filter(
            selector
).add_selection(selector)
co2_chart_full + netCO2_rule_chart

alt.LayerChart(...)

In [6]:
netCO2_Line

,CO2 emissions(T),Type
0,NaN,Original Net CO2 Emissions (World)


In [11]:
df = get_energy(with_coords=False)
cols = [col for col in df.columns if col not in ('Entity', 'Code', 'Year', 'TotalEnergy')]
df.head()

,Entity,Code,Year,Coal Consumption - TWh,Oil Consumption - TWh,Gas Consumption - TWh,Nuclear Consumption - TWh,Hydro Consumption - TWh,Wind Consumption - TWh,Solar Consumption - TWh,Geo Biomass Other - TWh,TotalEnergy
54,Africa,NaN,2019,1199.230132,2299.144350,1553.384218,33.666639,339.775310,46.695820,27.007849,8.228082,5507.132401
110,Algeria,DZA,2019,4.803190,237.003096,451.412500,0.000000,0.376385,0.024750,1.522873,0.000000,695.142794
166,Argentina,ARG,2019,8.123962,318.811166,465.594728,20.906696,91.785389,12.371094,1.980225,0.622945,920.196207
222,Asia Pacific,NaN,2019,33992.200917,19630.960314,8581.197775,1601.900933,4436.087853,1261.306200,970.773426,247.814646,70722.242063
278,Australia,AUS,2019,487.295356,585.703522,420.909709,0.000000,34.948995,48.216115,45.307300,3.467726,1625.848723


In [19]:
df_melt = df.melt(id_vars='Entity', value_vars=cols)
df_melt[df_melt.Entity=="India"]

,Entity,variable,value
35,India,Coal Consumption - TWh,5166.017819
129,India,Oil Consumption - TWh,2774.392986
223,India,Gas Consumption - TWh,592.503230
317,India,Nuclear Consumption - TWh,111.833093
411,India,Hydro Consumption - TWh,401.185055
505,India,Wind Consumption - TWh,156.779740
599,India,Solar Consumption - TWh,114.568842
693,India,Geo Biomass Other - TWh,29.667094


In [28]:
alt.Chart(
    df_melt, 
    title="Energy Mix for the Selected Country"
         ).mark_arc().encode(
    theta=alt.Theta(field="value"),
    color=alt.Color(field="variable", title="Fuel Type"),
    tooltip = [alt.Tooltip('value', title='Energy TWh'), alt.Tooltip('Entity', title='Country'), alt.Tooltip('variable', title='Fuel Type')],
    
)

alt.Chart(...)

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv('./data/Dataset1.csv')
df['Climate hazard'] = df['Climate hazard'].str.replace("[a-z0-9A-Z ]*> ", "")

<ipython-input-5-c7f6133887e1>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Climate hazard'] = df['Climate hazard'].str.replace("[a-z0-9A-Z ]*> ", "")


In [6]:
df.to_csv('./data/Dataset1_cleaned.csv')

In [4]:
cl.str.replace("[a-z0-9A-Z ]*> ", "")

<ipython-input-4-8ba3394c8747>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  cl.str.replace("[a-z0-9A-Z ]*> ", "")


0                   Landslide
1                   Heat wave
2                   Land fire
3                   Heat wave
4            Extreme hot days
                ...          
3540              Forest fire
3541               Rain storm
3542            Coastal flood
3543    Flash / surface flood
3544     Salt water intrusion
Name: Climate hazard, Length: 3545, dtype: object